<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/%E7%9C%BC%E7%90%83%E8%BD%AC%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import copy
import torch
import matplotlib.image as mpimg
import urllib.request
import zipfile
import os
import pandas as pd
from torchvision import datasets, transforms
import gdown

In [ ]:
train_csv_link = 'https://drive.google.com/uc?id=1SMb9CRHjB6UH2WnTZDFVSgpA6_nh75qN'

In [ ]:
data_dir = 'data'

In [ ]:
if not os.path.exists(data_dir):               
  os.makedirs(data_dir)

In [ ]:
dataset_url = "https://drive.google.com/uc?id=1G-4UhPKiQY3NxQtZiWuOkdocDTW6Bw0u"
zip_path = data_dir + 'images.zip'
gdown.download(dataset_url, zip_path, quiet=False)
print("Extracting...!")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
   zip_ref.extractall(data_dir)
print("Extracted!")
os.remove(zip_path)

Downloading...
From: https://drive.google.com/uc?id=1G-4UhPKiQY3NxQtZiWuOkdocDTW6Bw0u
To: /content/dataimages.zip
8.60GB [01:37, 88.3MB/s]


Extracting...!
Extracted!


In [ ]:
train_csv_link = 'https://drive.google.com/uc?id=1SMb9CRHjB6UH2WnTZDFVSgpA6_nh75qN'

In [ ]:
train_csv_path = data_dir + 'train_set.csv'
urllib.request.urlretrieve(train_csv_link, train_csv_path)

('datatrain_set.csv', <http.client.HTTPMessage at 0x7f52e63480d0>)

In [ ]:
tr_df = pd.read_csv(train_csv_path)

In [ ]:
tr_df['diagnosis'].value_counts()

0    1444
2     800
1     296
4     236
3     155
Name: diagnosis, dtype: int64

In [ ]:
df_class0=tr_df.loc[tr_df['diagnosis']==0]
df_class1=tr_df.loc[tr_df['diagnosis']==1]
df_class2=tr_df.loc[tr_df['diagnosis']==2]
df_class3=tr_df.loc[tr_df['diagnosis']==3]
df_class4=tr_df.loc[tr_df['diagnosis']==4]

In [ ]:
df1=df_class0.sample(len(df_class4))['id_code']
df1.reset_index(drop=True, inplace=True)
df2=df_class4['id_code']
df2.reset_index(drop=True, inplace=True)
df_0to4=pd.concat([df1,df2],axis=1) 

In [ ]:
df_0to4_tr=df_0to4[0:180]
df_0to4_val=df_0to4[180:210]
df_0to4_test=df_0to4[210:237]

In [ ]:
df_0to4_tr.columns = ['id_code_0', 'id_code_4']
df_0to4_val.columns = ['id_code_0', 'id_code_4']
df_0to4_test.columns = ['id_code_0', 'id_code_4']

In [ ]:
df_0to4_tr

,id_code_0,id_code_4
0,d866c26d76f0,001639a390f0
1,8ed586c43023,0083ee8054ee
2,540e4973829e,02685f13cefd
3,f4e68b61f480,02dda30d3acf
4,789c60cba801,0318598cfd16
...,...,...
175,4cfd22ae43d4,ba4d2c4b3039
176,51cd8d2057fa,bba38f2294a3
177,1fb455685328,bc34ed91c9bc
178,cfed7c1172ec,bd5013540a13


In [ ]:
os.mkdir("blind0to4")
os.mkdir("blind0to4/A/")
os.mkdir("blind0to4/B/")
os.mkdir("blind0to4/A/train/")
os.mkdir("blind0to4/A/val/")
os.mkdir("blind0to4/A/test/")
os.mkdir("blind0to4/B/train/")
os.mkdir("blind0to4/B/val/")
os.mkdir("blind0to4/B/test/")

In [ ]:
class0_train_path="blind0to4/A/train/"
class0_val_path="blind0to4/A/val/"
class0_test_path="blind0to4/A/test/"
class4_train_path="blind0to4/B/train/"
class4_val_path="blind0to4/B/val/"
class4_test_path="blind0to4/B/test/"

In [ ]:
original_imag_path= "data/images"

In [ ]:
import cv2
IMAGE_SIZE = (224, 224)
for index, row in df_0to4_tr.iterrows():
    class0_image_path = os.path.join(original_imag_path,row['id_code_0']+".png")
    class4_image_path = os.path.join(original_imag_path,row['id_code_4']+".png")
    class0_img=cv2.imread(class0_image_path)
    class4_img=cv2.imread(class4_image_path)

    class0_outimg=cv2.resize(class0_img,dsize=IMAGE_SIZE)
    class4_outimg=cv2.resize(class4_img,dsize=IMAGE_SIZE)
    
    cv2.imwrite(os.path.join(class0_train_path,str(index)+'.png'), class0_outimg)
    cv2.imwrite(os.path.join(class4_train_path,str(index)+'.png'), class4_outimg)

In [ ]:
IMAGE_SIZE = (224, 224)
for index, row in df_0to4_val.iterrows():
    class0_image_path = os.path.join(original_imag_path,row['id_code_0']+".png")
    class4_image_path = os.path.join(original_imag_path,row['id_code_4']+".png")
    class0_img=cv2.imread(class0_image_path)
    class4_img=cv2.imread(class4_image_path)

    class0_outimg=cv2.resize(class0_img,dsize=IMAGE_SIZE)
    class4_outimg=cv2.resize(class4_img,dsize=IMAGE_SIZE)
    
    cv2.imwrite(os.path.join(class0_val_path,str(index)+'.png'), class0_outimg)
    cv2.imwrite(os.path.join(class4_val_path,str(index)+'.png'), class4_outimg)

In [ ]:
IMAGE_SIZE = (224, 224)
for index, row in df_0to4_test.iterrows():
    class0_image_path = os.path.join(original_imag_path,row['id_code_0']+".png")
    class4_image_path = os.path.join(original_imag_path,row['id_code_4']+".png")
    class0_img=cv2.imread(class0_image_path)
    class4_img=cv2.imread(class4_image_path)

    class0_outimg=cv2.resize(class0_img,dsize=IMAGE_SIZE)
    class4_outimg=cv2.resize(class4_img,dsize=IMAGE_SIZE)
    
    cv2.imwrite(os.path.join(class0_test_path,str(index)+'.png'), class0_outimg)
    cv2.imwrite(os.path.join(class4_test_path,str(index)+'.png'), class4_outimg)

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')
!pip install -r requirements.txt


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2337, done.
remote: Total 2337 (delta 0), reused 0 (delta 0), pack-reused 2337
Receiving objects: 100% (2337/2337), 8.09 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (1499/1499), done.
     |████████████████████████████████| 686kB 11.6MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp37-none-any.whl size=655251 sha256=6b1adf829e764dad4700c6e140b95f3d9cef9936f2343ec1b17fa213e43f39fe
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=5b8d68ebf30feda0dc72eb615c9f33fc3b8f7f62e767086159cfcca3800e2544
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile


In [ ]:
!python datasets/combine_A_and_B.py --fold_A /content/blind0to4/A --fold_B /content/blind0to4/B --fold_AB /content/blind0to4

[fold_A] =  /content/blind0to4/A
[fold_B] =  /content/blind0to4/B
[fold_AB] =  /content/blind0to4
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = test, use 26/26 images
split = test, number of images = 26
split = train, use 180/180 images
split = train, number of images = 180
split = val, use 30/30 images
split = val, number of images = 30


In [ ]:
!python train.py --dataroot /content/blind0to4 --name class0to4_pix2pix --model pix2pix --direction AtoB

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/blind0to4            	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              

In [ ]:
!ls checkpoints/

class0to4_pix2pix  mosaic_pix2pix


In [ ]:
!python test.py --dataroot /content/blind0to4 --direction AtoB --model pix2pix --name class0to4_pix2pix

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /content/blind0to4            	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)